In [1]:
  %%javascript
    MathJax.Hub.Config({
      TeX: { equationNumbers: { autoNumber: "AMS" } }
    });

<IPython.core.display.Javascript object>

# Support Vector Machines
## Quadratic Programming Problem
There are a number of sources that will show that the first step to building a support vector machine is solving the following quadratic programming problem.

\begin{equation}
W(\alpha) = \sum_i^n \alpha_i - \sum_i^n\sum_j^n \alpha_i \alpha_j y_i y_j \vec{x}_i^T \vec{x}_j
\end{equation}
\begin{equation}
\sum_i^n \alpha_iy_i = 0, \quad \quad \alpha_i \geq 0
\end{equation}

where $\alpha_i$ is unknown, $y_i$ is the label, and $x_i$ is the data.

These equations have important characteristics to them. The most important are
\begin{equation} 
\vec{w} = \sum_i^n \alpha_i y_i x_i
\end{equation} where $w$ is a vector that helps to define our decision boundary and that most of the $\alpha_i$'s will be zero. So, most of the vectors that constitute $w$ will be 0.

Therefore, the points closed to our decision boundary will be used to define it.

## Kernel Trick
Equation (1) works simply in the case of a problem that is linearly seperable. However, in the case of classifying points that are not linearly seperable, we can use something called a kernel trick and modify equation (1) to be
\begin{equation}
W(\alpha) = \sum_i^n \alpha_i - \sum_i^n\sum_j^n \alpha_i \alpha_j y_i y_j K(\vec{x}_i,\vec{x}_j)
\end{equation}
where $K(\vec{x}_i, \vec{x}_j)$ is some transformation of $\vec{x}_i$ and $\vec{x}_j$. Some examples of common kernel tricks are

1. $(\vec{x}^T_i \vec{x}_j)^n$ where $n$ is any power.
1. $(\vec{x}^T_i \vec{x}_j + b)^n$ where $n$ is any power and $b$ is some bias.
1. $e^{||\vec{x}_i - \vec{x}_j||^2/\sigma^2}$ where $\sigma$ can be modified for various fitting.

The only requirement for a kernel trick is that it returns some numerical value that can be considered some kind of distance between points. That is, if $x_i$ and $x_j$ were images, there was some determination of a distance between the two images that comes out of this kernel trick.

## Algorithm Writing
Understanding these elements, we can begin to write a Support Vector Algorithm for classification. We will start with doing this under the pretense that all data is numerical in nature and so the kernel tricks needed will require no conversion to have numerical returns. 

The steps to writing a machine learning algorithm are

1. Train
1. Validate
1. Test

where using a training sample and equation (2) and (4), we come out with a model. That model is then validated against another sample. Several models will be built with various kernel tricks, and then they will also be validated. The model with the highest accuracy is then chosen and tested against our test set for accuracy and reported on.

#### Necessary Packages
Numpy: Numpy is needed for various matrix algebra and added mathematics functions.

In [2]:
import numpy as np
import numpy.linalg as la
import cvxopt
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import argh
import itertools

#### Kernel Functions
Before we are able to write a trainer, it is necessary to define a few kernel functions which will be a part of the Kernel class.

In [3]:
class Kernel(object):
    @staticmethod
    def linear():
        def f(x, y):
            return np.inner(x, y)
        return f
    
    @staticmethod
    def polynomial(dim, offset):
        def f(x, y):
            return (np.inner(x,y) + offset) ** dim
        return f
    
    @staticmethod
    def radial_basis(sigma):
        def f(x, y):
            num = la.norm(x - y) ** 2
            den = 2*sigma**2
            return np.exp(num/den)
        return f
        
    @staticmethod
    def gaussian(sigma):
        def f(x, y):
            num = la.norm(x-y) ** 2
            den = (2 * sigma ** 2)
            return np.exp(-np.sqrt(num/den))
        return f
    
    @staticmethod
    def sigmoid(gamma, offset):
        def f(x, y):
            return np.tanh(gamma * np.dot(x,y) + offset)
        return f

#### Writing a Predicter
We also need to have a predicter so that the trainer is able to continue updating itself. Predicting will use the decision boundary equation
\begin{equation}
\sum_i^n \alpha_i y_i K(\vec{x_i}, \vec{x}) + b \geq 0 \implies x \text{ is a positive sample.}
\end{equation}

In [4]:
class Predicter(object):
    def __init__(self, kernel, bias,
                alpha, X, labels):
        self._kernel = kernel
        self._bias = bias
        self._alpha = alpha
        self._X = X
        self._labels = labels
    def predict(self, x):
        result = self._bias
        for a_i, x_i, y_i in zip(self._alpha,
                                 self._X,
                                 self._labels):
            result += a_i*y_i*self._kernel(x_i, x)
        return np.sign(result).item()

This predicter object intializes itself as having the various components needed for labeling an individual point, and then has a function that performs the function. Then, depending on whether the sign is positive or negative, it returns a +1 for positive samples and a -1 for negative samples.

#### Writing a Trainer
Now is the significantly more difficult part. The trainer for SVM utilizes quadratic programming which is honestly something I'm not super familiar with, so I'll be utilizing the cvxopt package to find the proper $\alpha$'s to maximize $W(\alpha)$. I will also take a lot of (most of) inspiration from Andrew Tulloch's Trainer in his guide [here](http://tullo.ch/articles/svm-py/).

The following is a brief overview of each function within the Trainer class:

1. **__init__**: Initializes the trainer with a kernel function and the cost variable used to determine the accuracy of the quadratic programming maximization.
1. **train**: Computes the alphas (weights) and creates a predicter class for training iterations.
1. **compute_weights**: Computes the weights for the predicter by using quadratic programming to maximize $W(\alpha)$.
1. **create_predicter**: Creates predicter class by converting all below minimal weights to 0 and using all others as weights in predicter. This minimizes computation by only iterating over points with $\alpha > 0$. I also use Andrew Tulloch's code here to compute the bias which is based on a presentation from Carnegie Mellon.
1. **compute_gram**: This creates the Gram Matrix, which in Machine Learning is just a matrix of every $x_i, x_j$ pair in a kernel function where $G_{ij}$ = $K(x_i, x_j)$. The Gram Matrix is necessary for the quadratic programming maximization.

In [5]:
class Trainer(object):
    def __init__(self, kernel, cost):
        self._kernel = kernel
        self._c = cost
        
    def train(self, X, labels):
        weights = self.compute_weights(X, labels)
        return self.create_predicter(X, labels, weights)
    
    def compute_weights(self, X, labels):
        n, m = X.shape
        Gram = self.compute_gram(X)
        P = cvxopt.matrix(np.outer(labels, labels) * Gram)
        q = cvxopt.matrix(-1 * np.ones(n))

        # -a_i \leq 0
        # TODO(tulloch) - modify G, h so that we have a soft-margin classifier
        G_std = cvxopt.matrix(np.diag(np.ones(n) * -1))
        h_std = cvxopt.matrix(np.zeros(n))

        # a_i \leq c
        G_slack = cvxopt.matrix(np.diag(np.ones(n)))
        h_slack = cvxopt.matrix(np.ones(n) * self._c)

        G = cvxopt.matrix(np.vstack((G_std, G_slack)))
        h = cvxopt.matrix(np.vstack((h_std, h_slack)))

        A = cvxopt.matrix(labels, (1, n))
        b = cvxopt.matrix(0.0)

        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Lagrange multipliers
        return np.ravel(solution['x'])
    
    def create_predicter(self, X, labels, weights):
        non_minimal_indices = weights > 1e-5
        X_non_minimal = X[non_minimal_indices]
        labels_non_minimal = labels[non_minimal_indices]
        weights_non_minimal = weights[non_minimal_indices]

        bias = np.mean(
            [y_k - Predicter(
                kernel=self._kernel,
                bias=0.0,
                alpha=weights_non_minimal,
                X=X_non_minimal,
                labels=labels_non_minimal).predict(x_k)
            for (y_k, x_k) in zip(labels_non_minimal, X_non_minimal)])
        
        return Predicter(
                kernel = self._kernel,
                bias = bias,
                alpha = weights_non_minimal,
                X = X_non_minimal,
                labels = labels_non_minimal)
    
    
    def compute_gram(self, X):
        n, m = X.shape
        G = np.zeros((n, n))
        for i, x_i in enumerate(X):
            for j, x_j in enumerate(X):
                G[i, j] = self._kernel(x_i, x_j)
        return G

## Ex

In [6]:
def example(num_samples=500, num_features=2, grid_size=50, filename="svm.pdf"):
    samples = np.matrix(np.random.uniform(low = -1.5, high = 1.5, size=num_samples * num_features)
                        .reshape(num_samples, num_features))
    labels = 2 * (np.sqrt(np.power(samples[:,0],2) + np.power(samples[:,1],2)) > 1) - 1.0
    trainer = Trainer(Kernel.polynomial(2, 0), 0.01)
    predictor = trainer.train(samples, labels)

    plot(predictor, samples, labels, grid_size, filename)


def plot(predictor, X, y, grid_size, filename):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, grid_size),
                         np.linspace(y_min, y_max, grid_size),
                         indexing='ij')
    flatten = lambda m: np.array(m).reshape(-1,)

    result = []
    for (i, j) in itertools.product(range(grid_size), range(grid_size)):
        point = np.array([xx[i, j], yy[i, j]]).reshape(1, 2)
        result.append(predictor.predict(point))

    Z = np.array(result).reshape(xx.shape)

    plt.contourf(xx, yy, Z,
                 cmap=cm.Paired,
                 levels=[-0.001, 0.001],
                 extend='both',
                 alpha=0.8)
    plt.scatter(flatten(X[:, 0]), flatten(X[:, 1]),
                c=flatten(y), cmap=cm.Paired, edgecolor = "white",
                s = 9)
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.savefig("stuff.pdf")

    
argh.dispatch_command(example)

     pcost       dcost       gap    pres   dres
 0: -9.0145e+01 -1.0157e+01  3e+03  5e+01  5e-15
 1: -2.9069e+00 -1.0104e+01  4e+01  6e-01  5e-15
 2: -2.0133e+00 -6.8514e+00  5e+00  2e-16  1e-15
 3: -2.3218e+00 -2.9188e+00  6e-01  4e-17  9e-16
 4: -2.5902e+00 -2.6831e+00  9e-02  5e-17  8e-16
 5: -2.6241e+00 -2.6541e+00  3e-02  7e-17  8e-16
 6: -2.6321e+00 -2.6470e+00  1e-02  4e-17  7e-16
 7: -2.6362e+00 -2.6433e+00  7e-03  1e-16  7e-16
 8: -2.6389e+00 -2.6408e+00  2e-03  2e-16  9e-16
 9: -2.6397e+00 -2.6401e+00  4e-04  3e-16  9e-16
10: -2.6399e+00 -2.6399e+00  4e-06  2e-16  9e-16
11: -2.6399e+00 -2.6399e+00  4e-08  5e-17  9e-16
Optimal solution found.
